In [22]:
# importing the moduels
import pandas as pd
import requests
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt

In [23]:
# creating crime data frame
crime_df = pd.read_csv("Resources/Crime_Data_from_2020_to_present.csv")
crime_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277


In [24]:
crime_df.count()

DR_NO             966951
Date Rptd         966951
DATE OCC          966951
TIME OCC          966951
AREA              966951
AREA NAME         966951
Rpt Dist No       966951
Part 1-2          966951
Crm Cd            966951
Crm Cd Desc       966951
Mocodes           826575
Vict Age          966951
Vict Sex          833245
Vict Descent      833235
Premis Cd         966937
Premis Desc       966375
Weapon Used Cd    325476
Weapon Desc       325476
Status            966950
Status Desc       966951
Crm Cd 1          966940
Crm Cd 2           68656
Crm Cd 3            2302
Crm Cd 4              64
LOCATION          966951
Cross Street      150218
LAT               966951
LON               966951
dtype: int64

In [25]:
crime_df.fillna(0, inplace= True)


In [26]:
crime_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,0.0,0.0,1900 S LONGWOOD AV,0,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,0.0,0.0,1000 S FLOWER ST,0,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,0.0,0.0,0.0,1400 W 37TH ST,0,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,0.0,0.0,0.0,14000 RIVERSIDE DR,0,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,0.0,0.0,0.0,1900 TRANSIENT,0,34.0944,-118.3277


In [27]:
# cleaning the date reported and the date occured column to remove the 12:00:00
crime_df["Date Rptd"] = crime_df["Date Rptd"].str.split().str[0]
crime_df["DATE OCC"] = crime_df["DATE OCC"].str.split().str[0]
crime_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020,03/01/2020,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,0.0,0.0,1900 S LONGWOOD AV,0,34.0375,-118.3506
1,200106753,02/09/2020,02/08/2020,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,0.0,0.0,1000 S FLOWER ST,0,34.0444,-118.2628
2,200320258,11/11/2020,11/04/2020,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,0.0,0.0,0.0,1400 W 37TH ST,0,34.0210,-118.3002
3,200907217,05/10/2023,03/10/2020,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,0.0,0.0,0.0,14000 RIVERSIDE DR,0,34.1576,-118.4387
4,220614831,08/18/2022,08/17/2020,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,0.0,0.0,0.0,1900 TRANSIENT,0,34.0944,-118.3277


In [28]:
# cleaning the data to make the TIME OCC column read as 00:00 time structure
crime_df["TIME OCC"] = crime_df["TIME OCC"].astype(str).str.zfill(4).str[:2] + ":" + crime_df["TIME OCC"].astype(str).str.zfill(4).str[2:]
crime_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020,03/01/2020,21:30,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,0.0,0.0,1900 S LONGWOOD AV,0,34.0375,-118.3506
1,200106753,02/09/2020,02/08/2020,18:00,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,0.0,0.0,1000 S FLOWER ST,0,34.0444,-118.2628
2,200320258,11/11/2020,11/04/2020,17:00,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,0.0,0.0,0.0,1400 W 37TH ST,0,34.0210,-118.3002
3,200907217,05/10/2023,03/10/2020,20:37,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,0.0,0.0,0.0,14000 RIVERSIDE DR,0,34.1576,-118.4387
4,220614831,08/18/2022,08/17/2020,12:00,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,0.0,0.0,0.0,1900 TRANSIENT,0,34.0944,-118.3277


In [38]:
# creating a data frame with just selected information 
selected_columns = ["DR_NO", "Date Rptd", "DATE OCC", "TIME OCC", "Crm Cd"]
date_time_df = crime_df[selected_columns]
date_time_df.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,Crm Cd
0,190326475,03/01/2020,03/01/2020,21:30,510
1,200106753,02/09/2020,02/08/2020,18:00,330
2,200320258,11/11/2020,11/04/2020,17:00,480
3,200907217,05/10/2023,03/10/2020,20:37,343
4,220614831,08/18/2022,08/17/2020,12:00,354


In [39]:
# renaming the selceted columns to be more readable
date_time_df = date_time_df.rename(columns={"DR_NO": "Dr No"})
date_time_df.columns = date_time_df.columns.str.title()

In [46]:
# creating a new column called "Day of Week" from the "Date Reported" and "Date Occurred"
date_time_df["Date Rptd"] = pd.to_datetime(date_time_df["Date Rptd"])
date_time_df["Date Occ"] = pd.to_datetime(date_time_df["Date Occ"])
date_time_df["Day Of Wk Rptd"] = date_time_df["Date Rptd"].dt.day_name()
date_time_df["Day Of Wk Occ"] = date_time_df["Date Occ"].dt.day_name()
date_time_df["Year Rptd"] = date_time_df["Date Rptd"].dt.year
date_time_df["Year Occ"] = date_time_df["Date Occ"].dt.year
date_time_df

,Dr No,Date Rptd,Date Occ,Time Occ,Crm Cd,Day Of Wk Rptd,Day Of Wk Occ,Year Rptd,Year Occ
0,190326475,2020-03-01,2020-03-01,21:30,510,Sunday,Sunday,2020,2020
1,200106753,2020-02-09,2020-02-08,18:00,330,Sunday,Saturday,2020,2020
2,200320258,2020-11-11,2020-11-04,17:00,480,Wednesday,Wednesday,2020,2020
3,200907217,2023-05-10,2020-03-10,20:37,343,Wednesday,Tuesday,2023,2020
4,220614831,2022-08-18,2020-08-17,12:00,354,Thursday,Monday,2022,2020
...,...,...,...,...,...,...,...,...,...
966946,242004546,2024-01-16,2024-01-16,15:10,624,Tuesday,Tuesday,2024,2024
966947,240710284,2024-07-24,2024-07-23,14:00,510,Wednesday,Tuesday,2024,2024
966948,240104953,2024-01-15,2024-01-15,01:00,745,Monday,Monday,2024,2024
966949,241711348,2024-07-19,2024-07-19,07:57,888,Friday,Friday,2024,2024


In [51]:
date_time_df.to_csv("Resources/date_time.csv", index=False)

In [56]:
# need to split the data frame into one for each year (2020, 2021, 2022, 2023, 2024)

# setting the columns i want for the new data frame
columns = ["Dr No", "Date Rptd", "Date Occ", "Time Occ",
             "Year Occ", "Crm Cd", "Day Of Wk Rptd", 
            "Day Of Wk Occ"]

# creating the data frames based on year
crime_2020 = date_time_df.loc[date_time_df["Year Occ"] == 2020, columns]
crime_2021 = date_time_df.loc[date_time_df["Year Occ"] == 2021, columns]
crime_2022 = date_time_df.loc[date_time_df["Year Occ"] == 2022, columns]
crime_2023 = date_time_df.loc[date_time_df["Year Occ"] == 2023, columns]
crime_2024 = date_time_df.loc[date_time_df["Year Occ"] == 2024, columns]


In [62]:
crime_2021["Day Of Wk Occ"].value_counts()

Day Of Wk Occ
Friday       32358
Monday       30327
Saturday     30296
Wednesday    29431
Thursday     29418
Sunday       29383
Tuesday      28547
Name: count, dtype: int64

In [58]:
# creating csv files for the years 
crime_2020.to_csv("Resources/crime_2020.csv", index=False)
crime_2021.to_csv("Resources/crime_2021.csv", index=False)
crime_2022.to_csv("Resources/crime_2022.csv", index=False)
crime_2023.to_csv("Resources/crime_2023.csv", index=False)
crime_2024.to_csv("Resources/crime_2024.csv", index=False)

In [ ]:
#Clean descent data
clean_descent_df = crime_df[(crime_df["Vict Descent"] == 'H')]
clean_descent_df.count()